In [27]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
%matplotlib inline

## Class for the computation of the ESN activities

In [28]:
class Echo:
    
    def __init__(self,dt,tau_m,tau_M,diluition,N,W_in):
            
            self.dt=dt
            self.N=N
            self.tau_m=tau_m
            self.tau_M=tau_M
            self.Diluition=diluition
            
            W_np=np.random.uniform(-1,1,[N,N])
            D=np.random.uniform(0,1,(N,N))>np.ones((N,N))*diluition
            W_np=W_np*D.astype(int)
            
            eig=np.linalg.eigvals(W_np)
            self.eig=eig
            
            alpha_np=dt/(2*tau_m)
            pho_np=1-2*tau_m/tau_M
            
            W_np=pho_np*W_np/(np.max(np.absolute(eig)))
            
            self.W_np=W_np
            self.alpha_np=alpha_np
            self.pho_np=pho_np
            
            self.W=tf.Variable(W_np,trainable=False,dtype=tf.float32)
            self.alpha=tf.constant(alpha_np,dtype=tf.float32)
            
            self.W_in_np=W_in
            self.W_in=tf.Variable(W_in,trainable=False,dtype=tf.float32)
            self.N_in=np.shape(W_in)[0]
            
    def evolution_graph(self,T,init_state,inputs):

        
        state_hidden=init_state
        
        states_hidden=[]
        xs=[]
        
        for t in range(T):
            
            prev_state=tf.identity(state_hidden)
            state_hidden=( (1-self.alpha)*prev_state+self.alpha*tf.nn.relu( tf.matmul(prev_state,self.W)+tf.matmul(inputs[:,:,t],self.W_in) )) 
                        
            states_hidden.append(state_hidden)
            
            
        states=tf.concat([tf.expand_dims(s,2) for s in states_hidden],2)

    
        return state_hidden, states


## Definition of the input connectivity matrix

The weigths of the input connectivity follow a lognormal distribution. The number of connections to an input node is 
inversely proportional to their value...

In [29]:
def W_input(n_proj,N,n_mean_conn):
    
    m = 1
    v = 0.5
    mu = np.log((m**2)/np.sqrt(v+m**2))
    sigma = np.sqrt(np.log(v/(m**2)+1))
    X= np.random.lognormal(mu, sigma, N)

    alpha=np.mean(np.round(X))*n_mean_conn
    epsilon=0.3
    
    while np.mean(np.round(alpha/X))<n_mean_conn-epsilon or np.mean(np.round(alpha/X))>n_mean_conn+epsilon:

        if  np.mean(np.round(alpha/X))<n_mean_conn-epsilon:
            alpha=alpha+0.01;

        if  np.mean(np.round(alpha/X))>n_mean_conn+epsilon:
            alpha=alpha-0.01;


    Y=np.round(alpha/X).astype(int)
    W_In=np.zeros([N,N_proj])

    for i in range(N):
        W_In[i,np.random.randint(0,N_proj,Y[i])]=X[i]

    return W_In, X

## Definition of the input

Specifying N_basis and N_class, the following class creates N_class x N_class x N_basis x Sequence_lenght sequences.
The procedure will generate sequences and labels that will test the memory capacity of the network.
For a description of the procedure, see Appendix of the paper.


In [30]:
class input_stimuli:
    
    def __init__(self,T,N_basis,N_class,T_odour):
        
        import scipy.io
        odours = scipy.io.loadmat('hallem_olsen.mat')
        odours.keys()
        odours=odours['hallem_olsen']
        
        self.odours=odours/(np.max(np.max(odours,0)))                         # Normalisation of the data
        self.N_odours=np.shape(odours)[0]
        
        
        self.T=T
        self.N_class=N_class
                
        Sequence_length=3
        N_basis=N_class*N_basis
        
        
        N_sequence=(Sequence_length*N_class)*N_basis
        self.N_stimuli=N_sequence
        
        Sequence_basis=np.zeros([N_basis,Sequence_length])
        
        vector_index=np.ones([self.N_odours]);
        vector_index=vector_index.astype(int)
        
        # The loop defines, for each output class, a set of sequences (that in the paper we call 'context') composed by elements without repetitions 
        for i in range(N_class):
            
            random_int=np.random.permutation(self.N_odours)
        
            Sequence_basis[i*np.int(N_basis/N_class):(i+1)*np.int(N_basis/N_class),:]=np.reshape(random_int[0:Sequence_length*np.int(N_basis/N_class)],[np.int(N_basis/N_class),Sequence_length])
        
        
        Sequence=np.zeros([N_basis*Sequence_length*N_class,Sequence_length])
        Seq_basis=np.zeros([N_basis*Sequence_length*N_class,Sequence_length])
        
        # It replicates each sequence in the array Sequence_basis N_class*Sequence_length times to apply (later) 'perturbations' 
        # to its elements (see paper)  
        for i in range(np.shape(Sequence_basis)[0]):
            
            Seq_basis[N_class*Sequence_length*i:N_class*Sequence_length*i+N_class*Sequence_length,:]=Sequence_basis[i,:]
        
        Sequence=np.copy(Seq_basis)
        self.Sequence_basis=Seq_basis  
        
        ordered=np.linspace(0,self.N_odours-1,self.N_odours)
        permutated=np.random.permutation(ordered)
    
        # The multiples of N_split define when the perturbations are repeated
        N_split=int(np.shape(Seq_basis)[0]/N_class)
        
        # The loop applies 'perturbations' (change of elements) to the sequences 
        for i in range(int(N_basis/N_class)):
                                    
            
            for j in range(N_class*Sequence_length):
                
                # Index of the element that is changed
                h=np.floor(j/N_class).astype(int)
                
                for l in range(N_class):
                    
                    if l==0:
                        
                        Sequence[N_class*Sequence_length*i+j,h]=permutated[N_class*Sequence_length*i+j]
                    
                    # Repetitions of the perturbation 
                    Sequence[N_class*Sequence_length*i+j+N_split*l,h]=permutated[N_class*Sequence_length*i+j]
                
        Sequence=Sequence.astype(int)
        
        self.Sequence=Sequence        
        
        # Sequence is the array of sequences, it is a dictionary for the multi-d response of the input nodes.
        
        # Each element of each sequence corresponds to a vector in the odours array, containing the response of the input nodes
        # to the stimulus. Such vector is then is expanded for the amount of time T_odour that the NN is subjected to it.
        stimuli=np.zeros([self.N_stimuli,np.shape(odours)[1],Sequence_length*T_odour])
        for i in range(Sequence_length):
            
            stimuli[0:self.N_stimuli,0:np.shape(odours)[1],T_odour*i:T_odour*i+T_odour]=np.tile(np.expand_dims(self.odours[Sequence[:,i],:],2),[1,1,T_odour])
        
        
        self.stimuli=tf.constant(stimuli,dtype=tf.float32)
        
        # Definition of the desired output for the classification
        label=np.zeros([self.N_stimuli,N_class])
        
        # Array that helps to build the labels of the dataset of sequence
        label_basis=np.zeros([N_class,N_class,N_class])
        
        # Starting from an identity matrix, it builds the rest of the labels for  by shifting across the columns
        label_basis[:,:,0]=np.eye(N_class)
        for i in range(N_class-1):
            label_basis[:,:,i+1]=np.concatenate((label_basis[:,i+1:,0],label_basis[:,:i+1,0]),1)
        
        # It repeats the pattern of labels defined above for each set composed by N_split sequences (thus N_split/N_class times) 
        for i in range(N_class):
            label[i*N_split:i*N_split+N_split,:]=np.tile(label_basis[:,:,i],[int(N_split/N_class),1])
         
        self.labels=tf.constant(label,dtype=tf.float32)
        self.labels_np=label
        
        # Now that the labels are defined, we added multiplicative white noise to the data (the realisation of the noise 
        # is different for each sample). This is done in 'next_batch'...
        
    def next_batch(self,rand_ind,N_proj,sigma_noise):
                
        s_=tf.gather_nd(self.stimuli,rand_ind)
        
        s=s_+tf.random_normal(tf.shape(s_),0)*s_*sigma_noise
        
        labels=tf.gather_nd(self.labels,rand_ind)
        
        return s,labels
       

## Example of sequences of elements with the corresponding desired output

In the case below, the first three colums are the elements of the sequence, while the last two correspond to the desired output values

In [31]:
inp=input_stimuli(30,2,2,10)
print(np.concatenate([inp.Sequence, inp.labels_np],1))

[[131. 143.  35.   1.   0.]
 [ 81. 143.  35.   0.   1.]
 [113.  73.  35.   1.   0.]
 [113.  54.  35.   0.   1.]
 [113. 143.   0.   1.   0.]
 [113. 143.  68.   0.   1.]
 [110. 103. 146.   1.   0.]
 [ 47. 103. 146.   0.   1.]
 [ 54. 129. 146.   1.   0.]
 [ 54. 104. 146.   0.   1.]
 [ 54. 103.  84.   1.   0.]
 [ 54. 103. 154.   0.   1.]
 [131.  59. 159.   0.   1.]
 [ 81.  59. 159.   1.   0.]
 [172.  73. 159.   0.   1.]
 [172.  54. 159.   1.   0.]
 [172.  59.   0.   0.   1.]
 [172.  59.  68.   1.   0.]
 [110. 104.  27.   0.   1.]
 [ 47. 104.  27.   1.   0.]
 [ 39. 129.  27.   0.   1.]
 [ 39. 104.  27.   1.   0.]
 [ 39. 104.  84.   0.   1.]
 [ 39. 104. 154.   1.   0.]]


## Function for the computation of the specificity

The arguments are:
<ul>
    <li> state: the values of activities across the set of sequences considered (state has the dimensionality [Number of seq, Number of nodes]) </li>
    <li> labels: the desired output of the classification (dimensionality [Number of seq, Number of Classes]) </li>
    
The function will compute 2 specificity metrics, based on two different normalisations (see paper)


In [32]:
def specificity(state,labels):
    
    N_class=np.shape(labels)[1]
    N=np.shape(state)[1]
    N_test=np.shape(labels)[0]
    
    # It contains the number of times that a node is active for each class 
    activation_class=np.zeros([N,N_class]) 
    
    # For each separate class, the loop computes the number of times a node is active
    for i in range(N_class):
        
        # Filters the activity with the binary mask np.tile(np.expand_dims(labels[:,i],1),[1,N]) to consider only the sequences 
        # whose desired class is i 
        activity_class=state*np.tile(np.expand_dims(labels[:,i],1),[1,N])
        
        # Summation over the positive values across the first dimension, i.e. different sequences
        activation_class[:,i]=np.transpose(np.sum((activity_class>0),0))
            
    # Tensor describing the specificity
    spec_tensor=np.zeros([N,N_class,N_class])
    spec_tensor1=np.zeros([N,N_class,N_class])
    spec_tensor2=np.zeros([N,N_class,N_class])
    
    # Specificity
    spec=np.zeros([N])
    spec1=np.zeros([N])
    spec2=np.zeros([N])

    # Loop over each pair of classes 
    for i in range(N_class):
        
        for j in range(N_class):
            
            # Index to consider when a node is active at least once for the pair of classes considered
            index_help=activation_class[:,i]+activation_class[:,j]>0
            
            # Specificity, normalised with the total number of times the node was active
            spec_tensor[index_help,i,j]=np.absolute(activation_class[index_help,i]-activation_class[index_help,j])/(activation_class[index_help,i]+activation_class[index_help,j])
            
             # Specificity, normalised with the total number of sequences 
            spec_tensor1[index_help,i,j]=np.absolute(activation_class[index_help,i]-activation_class[index_help,j])/N_test
            

    for i in range(N):
        
        if np.sum(spec_tensor[i,:,:]>0)>0:
            
            # Average over the positive values in the specificity tensors
            spec[i]=np.mean(spec_tensor[i,spec_tensor[i,:,:]>0])
            spec1[i]=np.mean(spec_tensor1[i,spec_tensor[i,:,:]>0])

    return spec1, spec, spec_tensor, activation_class
    

## Tensorflow graph and training

The code will loop over the values of the variance of the external noise and the number of sequences specified.
For each pair of values in the grid search, it creates a tensorflow graph for the ESN and SpaRCe, which is composed by an initialisation phase (it initialises the starting value of the thresholds, theta_g) and a training phase (where the thresholds theta_i and the output weights W_out are optimised). The thresholds are theta=theta_g+theta_i, where the first factor corresponds to the initialisation, and the second factor is optimised (this split is due to convenience, to have the possibility to check each factor separately).  N_copies NNs, starting from different possible initial conditions, i.e. different values of theta_g, are then optimised.

In [34]:
T=30                        # Time steps in a sequence
N_class=2                   # Number of output classes
N=1000                      # Number of nodes in the ESN
T_odour=10                  # Temporal lenght of an element in the sequence

dt=0.01                     # time frame
tau_m=0.05                  # Minimum timescale in the ESN
tau_M=2                     # Maximum timescale in the ESN (the timescales of the ESN will be approximatly inside the interval
                            # [tau_m tau_M] )
diluition=0.99              # Probability of a zero in the connectivity matrix of the ESN
N_copies=10                 # Number of ESNs to be trained, starting from different initial conditions of the thresholds (theta_g)

N_episodes=70000            # Number of training instances
# Instances at which the code computes the performance
N_check_values=np.concatenate((np.arange(10000,N_episodes,10000),np.arange(61000,N_episodes+1000,1000)),axis=0)
N_check=np.shape(N_check_values)[0] # Number of instances at which the code computes the performance

alpha_size=0.002            # Learning rate
batch_size=30               

rescale_W_in=1              # Multiplicative factor of the input connectivity


# Values of the Parameters for the grid search (N_stimuli chenge controls the number of input sequences, that will be 
# N_stimuli_change[l]*N_class*N_class*Sequence_length, while sigma_noise_change controls the variance of the multiplicative white noise
# on the input data)

N_stimuli_change=[4,6,8,10,12,14,16,18,20,22,24,26,28]
sigma_noise_change=[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

thresholds_end=np.zeros([N_copies,N])

with tf.device('/GPU:0'):
    
    for k in range(np.shape(sigma_noise_change)[0]):
    
        for l in range(np.shape(N_stimuli_change)[0]):

            tf.reset_default_graph()

            N_basis=N_stimuli_change[l]
            sigma_noise=sigma_noise_change[k]
            
            inputs=input_stimuli(T,N_basis,N_class,T_odour)
            
            N_proj=np.shape(inputs.odours)[1]
            
            index_test_=np.arange(0,inputs.N_stimuli,dtype=int)
            N_epochs=10
            index_test_=np.tile(index_test_,[N_epochs])
            index_test=np.zeros([np.shape(index_test_)[0],1])
            index_test[:,0]=index_test_
            
            ## RNN DEFINITION AND TENSORFLOW GRAPH FOR THE ACTIVITY OF THE ESN AND SPARCE
            
            ## RNN DEFINITION
            
            W_in,X=W_input(N_proj,N,n_mean_conn=6)
            W_in=W_in.T/rescale_W_in

            rnn=Echo(dt,tau_m,tau_M,diluition,N,W_in)

            init_state=tf.placeholder(tf.float32,[None,N])
            ind_nextbatch=tf.placeholder(tf.int32,[None,1])

            stimuli,y_true=inputs.next_batch(ind_nextbatch,N_proj,sigma_noise)

            state,states=rnn.evolution_graph(T,init_state,stimuli)

            theta_g_start=np.zeros([N,N_copies])
            
            ## SPARCE
            
            # INITIALISATION OF THE THRESHOLDS
            # the values at which the thresholds are initialised are saved in the array theta_g, where each column contains 
            # a possible starting conditions. Different columns correspond to different percentiles of the activity distribution
            # of the nodes
            
            with tf.Session() as sess:

                sess.run(tf.global_variables_initializer())

                state_=sess.run(state,feed_dict={init_state:np.zeros([np.shape(index_test)[0],N]),ind_nextbatch:index_test})

                for i in range(N_copies):

                    theta_g=np.percentile(state_,i*N_copies,0)

                    x=(state_-theta_g)>0
                    theta_g_start[:,i]=theta_g

            theta_istart=np.random.randn(N)/N
            
            R=10*y_true
            
            # TRAINING
            
            # The following lines define N_copies NNs with different thresholds values, i.e. starting from different initial 
            # conditions (the diverse values of theta_g). Training of output weights and thresholds between different NNs is 
            # independent, but they share the same representation of activities (called state above, V in the paper)
            
            theta_g=[]
            theta_i=[]
            W_out=[]
            state_sparse=[]
            y=[]
            error=[]
            train1=[]
            train2=[]
            choices=[]
            actions=[]
            p=[]

            for i in range(N_copies):

                theta_g.append(tf.Variable(np.expand_dims(theta_g_start[:,i],0), trainable=False, dtype=tf.float32))

                theta_i.append(tf.Variable(theta_istart,dtype=tf.float32))

                W_out.append(tf.Variable(np.random.uniform(0,1,[N,N_class])/(N/10),dtype=tf.float32))

                state_sparse.append(tf.nn.relu(state-theta_g[i]-theta_i[i]))               

                y.append(tf.matmul(state_sparse[i],W_out[i]))

                error.append(tf.reduce_mean(tf.square(R-y[i])))

                train1.append(tf.train.GradientDescentOptimizer(learning_rate=alpha_size).minimize(error[i],var_list=[W_out[i]]))
                train2.append(tf.train.GradientDescentOptimizer(learning_rate=alpha_size/10).minimize(error[i],var_list=[theta_i[i]]))

                train=train1+train2

                choices.append(tfp.distributions.OneHotCategorical(probs=tf.nn.softmax(y[i])))
                actions.append(tf.cast(choices[i].sample(),tf.float32))

            ## END OF THE TENSORFLOW GRAPH
            
            # VARIABLES SAVED DURING TRAINING
            
            theta_save=np.zeros([N,N_check])

            gradients_W=np.zeros([N,N_class,N_episodes])
            gradients_theta=np.zeros([N,N_episodes])
            
            coding_levels=np.zeros([N_copies,N_check])
            performance=np.zeros([N_copies,N_check])
            costs=np.zeros([N_copies,N_check])
            spec_save=np.zeros([N_copies,N_check])
            spec1_save=np.zeros([N_copies,N_check])

            init=tf.global_variables_initializer()

            with tf.Session() as sess:

                sess.run(init)
                
                q=0
                
                for n in range(N_episodes+1):

                    ind_next=np.random.randint(0,inputs.N_stimuli,[batch_size,1])

                    _=sess.run([t for t in train],feed_dict={init_state:np.zeros([batch_size,N]),ind_nextbatch:ind_next})

                    if n==N_check_values[q]:
                        
                        index_help=np.copy(q)

                        q=q+1
                        
                        for i in range(N_copies):

                            matches=tf.equal(tf.argmax(y_true,1),tf.argmax(y[i],1))
                            p=tf.reduce_mean(tf.cast(matches,tf.float32))

                            labels_,state_,p_ra,error_=sess.run([y_true,state_sparse[i],p,error[i]],feed_dict={init_state:np.zeros([np.shape(index_test)[0],N]),ind_nextbatch:index_test})

                            coding_level=np.mean(state_>0)
                            spec1, spec, spec_tensor, activation_class=specificity(state_,labels_)

                            coding_levels[i,index_help]=np.copy(coding_level)
                            performance[i,index_help]=np.copy(np.array(p_ra))
                            costs[i,index_help]=np.copy(error_)
                            spec1_save[i,index_help]=np.mean(spec1)
                            spec_save[i,index_help]=np.mean(spec)
                            
                            
                            print('Iteration: ',n,'Reservoir ',i,'Probability: ',p_ra,'Error: ',error_,'Coding: ',coding_level,'Spec: ',np.mean(spec),' ',np.mean(spec1))
                        
                        
                        if n==N_check_values[-1]:


                            print('End simulation and saving')

                            
                            #np.savetxt('coding_levels_eta002_batch30_'+repr(N_stimuli_change[l])+'_'+repr(sigma_noise_change[k]),coding_levels)
                            #np.savetxt('performance_eta002_batch30_'+repr(N_stimuli_change[l])+'_'+repr(sigma_noise_change[k]),performance)
                            #np.savetxt('error_eta002_batch30_'+repr(N_stimuli_change[l])+'_'+repr(sigma_noise_change[k]),costs)
                            #np.savetxt('spec1_eta002_batch30_'+repr(N_stimuli_change[l])+'_'+repr(sigma_noise_change[k]),spec1_save)  
                            #np.savetxt('spec2_eta002_batch30_'+repr(N_stimuli_change[l])+'_'+repr(sigma_noise_change[k]),spec2_save)  
                            #np.savetxt('spec_eta002_batch30_'+repr(N_stimuli_change[l])+'_'+repr(sigma_noise_change[k]),spec_save)  

        


Iteration:  10000 Reservoir  0 Probability:  0.6875 Error:  18.70418 Coding:  0.9743958333333333 Spec:  0.010683858431016496   0.010354166666666668
Iteration:  10000 Reservoir  1 Probability:  1.0 Error:  9.119197 Coding:  0.88475 Spec:  0.014818146788030522   0.012833333333333332
Iteration:  10000 Reservoir  2 Probability:  1.0 Error:  5.964923 Coding:  0.78725 Spec:  0.021074006151104954   0.01633333333333333
Iteration:  10000 Reservoir  3 Probability:  1.0 Error:  4.5380335 Coding:  0.6854166666666667 Spec:  0.026258329120673395   0.017625
Iteration:  10000 Reservoir  4 Probability:  1.0 Error:  3.8438635 Coding:  0.5897916666666667 Spec:  0.03637073358433778   0.02083333333333333
Iteration:  10000 Reservoir  5 Probability:  1.0 Error:  3.719245 Coding:  0.5069166666666667 Spec:  0.04012935735822512   0.01983333333333333
Iteration:  10000 Reservoir  6 Probability:  1.0 Error:  3.6830826 Coding:  0.427125 Spec:  0.048934495896868   0.020166666666666663
Iteration:  10000 Reservoir  7 

Iteration:  60000 Reservoir  9 Probability:  1.0 Error:  0.6401461 Coding:  0.17777083333333332 Spec:  0.16366689097659917   0.0199375
Iteration:  61000 Reservoir  0 Probability:  1.0 Error:  1.8483508 Coding:  0.9528541666666667 Spec:  0.011371592712285265   0.010354166666666666
Iteration:  61000 Reservoir  1 Probability:  1.0 Error:  0.52878416 Coding:  0.8674375 Spec:  0.019266128543304326   0.0158125
Iteration:  61000 Reservoir  2 Probability:  1.0 Error:  0.19794764 Coding:  0.7764583333333334 Spec:  0.028899904780882352   0.021375
Iteration:  61000 Reservoir  3 Probability:  1.0 Error:  0.116691366 Coding:  0.6798541666666666 Spec:  0.03529963235308118   0.0230625
Iteration:  61000 Reservoir  4 Probability:  1.0 Error:  0.08391023 Coding:  0.5831666666666667 Spec:  0.04418955285717702   0.024416666666666663
Iteration:  61000 Reservoir  5 Probability:  1.0 Error:  0.1029995 Coding:  0.506875 Spec:  0.05245411859380232   0.02520833333333333
Iteration:  61000 Reservoir  6 Probabilit

Iteration:  66000 Reservoir  9 Probability:  1.0 Error:  0.53157353 Coding:  0.17804166666666665 Spec:  0.16574286096421167   0.020208333333333335
Iteration:  67000 Reservoir  0 Probability:  1.0 Error:  1.5588008 Coding:  0.9517083333333334 Spec:  0.012347801318862046   0.011375
Iteration:  67000 Reservoir  1 Probability:  1.0 Error:  0.41712224 Coding:  0.8665625 Spec:  0.020263690178124877   0.016645833333333332
Iteration:  67000 Reservoir  2 Probability:  1.0 Error:  0.15112545 Coding:  0.7765 Spec:  0.02972782925660012   0.022
Iteration:  67000 Reservoir  3 Probability:  1.0 Error:  0.083266325 Coding:  0.6799583333333333 Spec:  0.03581103079473026   0.023375
Iteration:  67000 Reservoir  4 Probability:  1.0 Error:  0.05721238 Coding:  0.5832916666666667 Spec:  0.044395036870910606   0.024583333333333332
Iteration:  67000 Reservoir  5 Probability:  1.0 Error:  0.071936876 Coding:  0.5065833333333334 Spec:  0.05192825540132361   0.024916666666666663
Iteration:  67000 Reservoir  6 Pr

Iteration:  20000 Reservoir  8 Probability:  1.0 Error:  3.3256564 Coding:  0.26905555555555555 Spec:  0.06653820704879772   0.015305555555555557
Iteration:  20000 Reservoir  9 Probability:  0.9861111 Error:  5.0257845 Coding:  0.17968055555555557 Spec:  0.09589820071941106   0.013402777777777779
Iteration:  30000 Reservoir  0 Probability:  0.9166667 Error:  10.903608 Coding:  0.9675555555555555 Spec:  0.006288122322117814   0.005861111111111111
Iteration:  30000 Reservoir  1 Probability:  1.0 Error:  2.5533075 Coding:  0.8633194444444444 Spec:  0.014434076741031605   0.011958333333333335
Iteration:  30000 Reservoir  2 Probability:  1.0 Error:  1.3546501 Coding:  0.7665416666666667 Spec:  0.019340454055921948   0.014291666666666668
Iteration:  30000 Reservoir  3 Probability:  1.0 Error:  1.015884 Coding:  0.6842083333333333 Spec:  0.024301005385275624   0.01576388888888889
Iteration:  30000 Reservoir  4 Probability:  1.0 Error:  0.81538814 Coding:  0.6007361111111111 Spec:  0.029341813

Iteration:  62000 Reservoir  5 Probability:  1.0 Error:  0.15165123 Coding:  0.5141805555555555 Spec:  0.04163055685263017   0.020097222222222225
Iteration:  62000 Reservoir  6 Probability:  1.0 Error:  0.17157966 Coding:  0.4330555555555556 Spec:  0.05006062315970749   0.019805555555555555
Iteration:  62000 Reservoir  7 Probability:  1.0 Error:  0.252043 Coding:  0.35291666666666666 Spec:  0.06257122180365085   0.01922222222222222
Iteration:  62000 Reservoir  8 Probability:  1.0 Error:  0.45820746 Coding:  0.2753611111111111 Spec:  0.08100298125111204   0.017694444444444447
Iteration:  62000 Reservoir  9 Probability:  1.0 Error:  0.7934534 Coding:  0.18783333333333332 Spec:  0.12071334319708774   0.016222222222222225
Iteration:  63000 Reservoir  0 Probability:  1.0 Error:  3.6363275 Coding:  0.9581666666666667 Spec:  0.006346099094417102   0.005694444444444445
Iteration:  63000 Reservoir  1 Probability:  1.0 Error:  0.5288942 Coding:  0.8581666666666666 Spec:  0.017070140200393616   0

Iteration:  68000 Reservoir  3 Probability:  1.0 Error:  0.11208712 Coding:  0.6828611111111111 Spec:  0.028657710157496744   0.018305555555555558
Iteration:  68000 Reservoir  4 Probability:  1.0 Error:  0.09721813 Coding:  0.6028472222222222 Spec:  0.034910077786816436   0.01981944444444445
Iteration:  68000 Reservoir  5 Probability:  1.0 Error:  0.11201086 Coding:  0.5145972222222223 Spec:  0.04304018960543941   0.020736111111111115
Iteration:  68000 Reservoir  6 Probability:  1.0 Error:  0.12940659 Coding:  0.43377777777777776 Spec:  0.05076030129315077   0.020166666666666673
Iteration:  68000 Reservoir  7 Probability:  1.0 Error:  0.1974436 Coding:  0.35305555555555557 Spec:  0.06370400289059935   0.01952777777777778
Iteration:  68000 Reservoir  8 Probability:  1.0 Error:  0.3756444 Coding:  0.27547222222222223 Spec:  0.08391193649445394   0.018305555555555558
Iteration:  68000 Reservoir  9 Probability:  1.0 Error:  0.6622065 Coding:  0.18813888888888888 Spec:  0.12211827465315744 

Iteration:  40000 Reservoir  1 Probability:  1.0 Error:  2.4405656 Coding:  0.8615416666666667 Spec:  0.011515128884673997   0.0093125
Iteration:  40000 Reservoir  2 Probability:  1.0 Error:  1.2411062 Coding:  0.7673125 Spec:  0.016170438927753942   0.011833333333333333
Iteration:  40000 Reservoir  3 Probability:  1.0 Error:  0.82213503 Coding:  0.6878958333333334 Spec:  0.018903676949514604   0.01225
Iteration:  40000 Reservoir  4 Probability:  1.0 Error:  0.67887104 Coding:  0.6011666666666666 Spec:  0.023717966992814234   0.0135
Iteration:  40000 Reservoir  5 Probability:  1.0 Error:  0.67194283 Coding:  0.5160416666666666 Spec:  0.028511958292855053   0.013583333333333333
Iteration:  40000 Reservoir  6 Probability:  1.0 Error:  0.8022757 Coding:  0.43335416666666665 Spec:  0.03315200748272404   0.0131875
Iteration:  40000 Reservoir  7 Probability:  1.0 Error:  1.0968629 Coding:  0.3514791666666667 Spec:  0.04283729198965767   0.013479166666666665
Iteration:  40000 Reservoir  8 Pro

Iteration:  63000 Reservoir  9 Probability:  1.0 Error:  1.4805206 Coding:  0.19139583333333332 Spec:  0.08850090693583235   0.011645833333333333
Iteration:  64000 Reservoir  0 Probability:  1.0 Error:  5.7674685 Coding:  0.96125 Spec:  0.005898708711895927   0.005354166666666666
Iteration:  64000 Reservoir  1 Probability:  1.0 Error:  0.74494106 Coding:  0.85959375 Spec:  0.013409869756617156   0.010739583333333332
Iteration:  64000 Reservoir  2 Probability:  1.0 Error:  0.29202288 Coding:  0.7668020833333333 Spec:  0.0178504733873947   0.01296875
Iteration:  64000 Reservoir  3 Probability:  1.0 Error:  0.15341707 Coding:  0.6868020833333334 Spec:  0.02129393348164256   0.013593749999999998
Iteration:  64000 Reservoir  4 Probability:  1.0 Error:  0.12821902 Coding:  0.60025 Spec:  0.026529356975205868   0.014958333333333332
Iteration:  64000 Reservoir  5 Probability:  1.0 Error:  0.13440189 Coding:  0.5155 Spec:  0.03135635079432106   0.0148125
Iteration:  64000 Reservoir  6 Probabili

Iteration:  69000 Reservoir  8 Probability:  1.0 Error:  0.33362675 Coding:  0.27902083333333333 Spec:  0.05686901209641137   0.0134375
Iteration:  69000 Reservoir  9 Probability:  1.0 Error:  1.1759813 Coding:  0.1923125 Spec:  0.088258389337204   0.0115


KeyboardInterrupt: 